In [ ]:
%reload_ext autoreload
%autoreload 2

import seaborn as sns

import sys
sys.path.append(os.path.abspath('../'))
from common import *
import configs
import opt_configs

from constants import OUTPUT_DIR
import matplotlib.gridspec as gridspec

def gallery(array, ncols=2):
    """Code from https://stackoverflow.com/a/42041135/2351867"""
    nindex, height, width, intensity = array.shape
    nrows = nindex // ncols
    assert nindex == nrows*ncols
    result = (array.reshape(nrows, ncols, height, width, intensity)
              .swapaxes(1, 2)
              .reshape(height*nrows, width*ncols, intensity))
    return result

def set_spine_color(ax, color='white'):
    ax.spines['bottom'].set_color(color)
    ax.spines['top'].set_color(color) 
    ax.spines['right'].set_color(color)
    ax.spines['left'].set_color(color)


darkmode = False
write_png = False

linecolor = 'white' if darkmode else 'black'

sns.set()
sns.set_style('darkgrid', {'axes.edgecolor': 'black'})
matplotlib.rcParams['axes.linewidth'] = 0.5

fig_name = 'teaser'
fig_dir = join(FIGURE_DIR, fig_name)
used_configs = [configs.Warp()]
names = ['Ours']

scenes = []
used_opt_configs = []
names_pretty = []
insets = []

scenes.append('bench')
used_opt_configs.append('diffuse-32-hq')

show_iters = ['final']
ref_views = [0, 4, 8, 10]

exposure = 0.5
fontsize = 16
base_size = 2
n_rows = len(scenes)
n_cols = len(show_iters) + 3
total_width = base_size * n_cols
aspect = 0.98 * n_rows / n_cols
aspect = 0.42
total_width = TEXT_WIDTH
fig = plt.figure(1, figsize=(total_width, aspect * total_width), constrained_layout=False)
iter_rename_dict = {show_iters[-1]: 'Final', 0: 'Initialization'}

gs = fig.add_gridspec(2, 1, wspace=0.0, hspace=0.0, height_ratios=[2, 1])

gs0 = gridspec.GridSpecFromSubplotSpec(1, len(show_iters) + 3, subplot_spec=gs[0], wspace=0.025, hspace=0.0)
inset_size = 0.2
y_offset = -0.05
crop_pixels = 40
ranges = [5e-7, 2e-6, 5e-5]
scene = scenes[0]
opt_name = used_opt_configs[0]

opt_config = opt_configs.get_opt_config(opt_name)
output_dir = join(OUTPUT_DIR, scene, opt_config.name, used_configs[0].name) 

show_ref_as_subplots = False
if show_ref_as_subplots:
    gs_insets = gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=gs0[0], wspace=0.0, hspace=0.0)
all_ref_images = []
for idx, ref_view in enumerate(ref_views):
    r = idx // 2
    c = idx % 2
    img = read_img(join(output_dir, f'ref-{ref_view:02d}.exr'), exposure=exposure)
    if all_ref_images is None:
        all_ref_images = np.zeros((2 * img.shape[0], 2 * img.shape[1], 3))
    all_ref_images.append(img)

    if show_ref_as_subplots:
        ax = fig.add_subplot(gs_insets[r, c])
        ax.imshow(img, interpolation='none')
        disable_ticks(ax)

ax = fig.add_subplot(gs0[0])
txt = ax.set_title('(a) Reference images', color=linecolor, fontsize=fontsize, y=y_offset, va='top')

if not show_ref_as_subplots:
    all_ref_images = gallery(np.stack(all_ref_images, axis=0))
    all_ref_images[all_ref_images.shape[0]//2, :, :] = 1
    all_ref_images[:, all_ref_images.shape[1]//2, :] = 1
    ax.imshow(all_ref_images)
    disable_ticks(ax)
    set_spine_color(ax, color=linecolor)
else:
    ax.set_axis_off()

col = 1
r = None
for cfg in used_configs:
    ax = fig.add_subplot(gs0[col])
    fn = join(FIGURE_DIR, fig_name, scene, f'warp_final_geo.exr')
    img = read_img(fn, exposure=exposure)
    img = img[crop_pixels:-crop_pixels, crop_pixels:-crop_pixels, :]
    ax.imshow(img, interpolation='none', extent=[0, 1, 0, 1])
    disable_ticks(ax)
    txt = ax.set_title('(b) Optimized surface', color=linecolor, fontsize=fontsize, y=y_offset, va='top')
    col += 1
    set_spine_color(ax, color=linecolor)

    ax = fig.add_subplot(gs0[col])
    fn = join(FIGURE_DIR, fig_name, scene, f'{cfg.name}_final.exr')
    img = read_img(fn, exposure=exposure)
    img = img[crop_pixels:-crop_pixels, crop_pixels:-crop_pixels, :]
    ax.imshow(img, interpolation='none', extent=[0, 1, 0, 1])
    disable_ticks(ax)
    txt = ax.set_title('(c) Optimization result', color=linecolor, fontsize=fontsize, y=y_offset, va='top')
    col += 1
    set_spine_color(ax, color=linecolor)

    ax = fig.add_subplot(gs0[col])
    fn = join(FIGURE_DIR, fig_name, scene, f'reference.exr')
    img = read_img(fn, exposure=exposure)
    img = img[crop_pixels:-crop_pixels, crop_pixels:-crop_pixels, :]
    ax.imshow(img, interpolation='none', extent=[0, 1, 0, 1])
    disable_ticks(ax)
    txt = ax.set_title('(d) Ground truth', color=linecolor, fontsize=fontsize, y=y_offset, va='top')
    col += 1
    set_spine_color(ax, color=linecolor)

# Add optimization states
if True:
    show_iters = np.arange(0, 512, 64).tolist() + [511,]
    gs1 = gridspec.GridSpecFromSubplotSpec(1, len(show_iters), subplot_spec=gs[1], wspace=0.025, hspace=0.0)
    scene = scenes[0]
    opt_name = used_opt_configs[0]

    opt_config = opt_configs.get_opt_config(opt_name)
    output_dir = join(OUTPUT_DIR, scene, opt_config.name, used_configs[0].name)
    cfg = configs.Warp()
    r = None

    iter_rename_dict = {show_iters[-1]: '512', 0: 'Initialization'}
    for col, show_iter in enumerate(show_iters):
        ax = fig.add_subplot(gs1[col])
        suffix = f'{show_iter:04d}' if isinstance(show_iter, int) else show_iter

        fn = join(FIGURE_DIR, fig_name, scene, f'{cfg.name}_{suffix}.exr')
        img = read_img(fn, exposure=exposure)
        topc = 40
        bottomc = 2 * crop_pixels - topc
        img = img[topc:-bottomc, crop_pixels:-crop_pixels, :]
        ax.imshow(img, interpolation='none', extent=[0, 1, 0, 1])
        disable_ticks(ax)
        set_spine_color(ax, color=linecolor)

        label = iter_rename_dict[show_iter] if show_iter in iter_rename_dict.keys() else f'{show_iter}'
        txt = ax.set_title(label, y=0.9, x=0.025, color='k', ha='left', va='top', fontsize=14)
        txt.set_path_effects([path_effects.withStroke(linewidth=0.5, foreground='white')])
plt.margins(0, 0)

if write_png:
    kwargs = {'file_format': 'png', 'transparent': True, 'dpi': 250}

# save_fig(fig_name, **kwargs)
